In [9]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime
import glob

In [18]:
parentDir = os.path.abspath(os.path.join(os.getcwd(),'..'))
os.chdir(parentDir)

In [3]:
from support_functions import portfolio, data_loader

In [4]:
importlib.reload(data_loader)

<module 'support_functions.data_loader' from '/Users/yifanli/Github/fidelity-portfolio-tracker/code/support_functions/data_loader.py'>

# Full Run

In [6]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'

transactions = data_loader.load_transaction(data_folder_path, transaction_file_pattern)
position = data_loader.load_position(data_folder_path, position_file_pattern)

ValueError: No objects to concatenate

# Check data_loader

## Check load_transaction

In [57]:
def _gather_transaction_files(data_folder_path,transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)
    return transaction_files

def _combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions

def _remove_NA_value(df,colName):
    df_copy = df.copy()
    df_copy = df_copy[
        df_copy[colName].notna()
    ] 
    return df_copy

def _remove_leading_space(df,colName):
    df_copy = df.copy()
    df_copy[colName] = df_copy[colName].str.lstrip()
    return df_copy

def _str_to_date(df, colName, format):
    df_copy = df.copy()
    df_copy[colName] = pd.to_datetime(
        df_copy[colName], format=format
    ).dt.date
    return df_copy

def _add_Transfer_symbol(df):
    df_copy = df.copy()
    df_copy.loc[df_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    return df_copy

def _sort_df_by_column(df, colName):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(by=colName).reset_index(
        drop=True
    )
    return df_copy


def _clean_transactions(transactions):
    transactions = _remove_NA_value(transactions,"Amount ($)")
    transactions = _remove_leading_space(transactions,"Run Date")
    transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
    transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
    transactions = _add_Transfer_symbol(transactions)
    transactions = _remove_leading_space(transactions,"Symbol")
    transactions = _sort_df_by_column(transactions,"Run Date")
    return transactions

In [58]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [59]:
transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
transactions = _combine_transaction_files(transaction_files)
transactions = _clean_transactions(transactions)

In [53]:
transactions = _remove_NA_value(transactions,"Amount ($)")
transactions = _remove_leading_space(transactions,"Run Date")
transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
transactions = _add_Transfer_symbol(transactions)
transactions = _remove_leading_space(transactions,"Symbol")
transactions = _sort_df_by_column(transactions,"Run Date")

## Check load_position

In [ ]:
def _find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file

def _gather_position_files(data_folder_path, position_file_pattern):
    position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
    position_files = glob.glob(position_file_path_pattern)
    position_file = _find_latest_position_file(position_files)
    return position_file

def _transfer_dollar_to_float(df, colNames):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    df_copy = df.copy()
    # Replace any "--" with "$0"
    cleaned = df_copy[colNames].str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    df_copy[colNames] = cleaned.astype(float)
    return df_copy

def _clean_position(position):
    position = _remove_NA_value(position,"Current Value")
    position = _transfer_dollar_to_float(position, "Current Value")
    position = _transfer_dollar_to_float(position, "Cost Basis Total")
    return position

In [71]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [73]:
position_file = _gather_position_files(data_folder_path, position_file_pattern)
position_file

'./data/Portfolio_Positions_Aug-05-2025.csv'

In [74]:
position = pd.read_csv(position_file)

In [75]:
position = _remove_NA_value(position,"Current Value")
position = _transfer_dollar_to_float(position, "Current Value")
position = _transfer_dollar_to_float(position, "Cost Basis Total")

# Attribute 1: get investment distribution

In [6]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

NameError: name 'transactions' is not defined

In [7]:
current_portfolio.show_investment_distribution()

   Class     Amount  Percent
0  stock  240727.72   40.12%
1   bill  345920.55   57.65%
2  other   13351.73    2.23%
3  total  600000.00  100.00%


# Attribute 2: get IRR for each stock

In [10]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [11]:
current_portfolio.show_stock_irr()

       Stock      irr
0   Transfer   11.57%
1       AAPL   19.95%
2       SBUX   14.71%
3        JPM   39.95%
4        AXP   35.91%
5      GOOGL   47.53%
6       AMZN   59.82%
7        NKE  -10.67%
8       TSLA  101.81%
9      FXAIX   30.93%
10       MCD   35.55%
11     FSKAX   34.87%
12     FSPSX   -5.18%
13      COKE   41.59%
14      MSFT  -12.08%
15      BRKB   30.43%
16     stock   23.08%


# Attribute 3: get irr of each portfolio

In [18]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [20]:
current_portfolio.pension_transactions
# current_portfolio.transactions[
#             current_portfolio.transactions["Account"] == "ERNST & YOUNG 401(K) 86964"
#         ].shape

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
103,2024-01-26,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.070,NaN,NaN,NaN,NaN,242.30,NaT
104,2024-01-26,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.716,NaN,NaN,NaN,NaN,103.85,NaT
114,2024-02-09,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.041,NaN,NaN,NaN,NaN,242.30,NaT
115,2024-02-09,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.542,NaN,NaN,NaN,NaN,103.85,NaT
122,2024-02-23,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.493,NaN,NaN,NaN,NaN,103.85,NaT
123,2024-02-23,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.027,NaN,NaN,NaN,NaN,242.30,NaT
134,2024-03-08,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.408,NaN,NaN,NaN,NaN,103.85,NaT
135,2024-03-08,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.020,NaN,NaN,NaN,NaN,242.30,NaT
138,2024-03-22,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.336,NaN,NaN,NaN,NaN,103.85,NaT
139,2024-03-22,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,0.997,NaN,NaN,NaN,NaN,242.30,NaT


In [267]:
current_portfolio.add_total_current_value_to_individual_position()
current_portfolio.set_merged_individual_position_transaction()
current_portfolio.add_time_diff_in_merged_individual_position_transaction()
unique_symbols = current_portfolio.merged_individual_position_transaction["Symbol"].unique()
stock_list = [
    element
    for element in unique_symbols
    if "FZFXX" not in element and not element[0].isdigit()
]
stock_list

Total current value has been added


['Transfer',
 'AAPL',
 'SBUX',
 'JPM',
 'AXP',
 'GOOGL',
 'AMZN',
 'NKE',
 'TSLA',
 'FXAIX',
 'MCD',
 'FSKAX',
 'FSPSX',
 'COKE',
 'MSFT',
 'BRKB',
 'Pending Activity']